### Preprocess한 Maestro dataset .npy 파일들을 다운로드 합니다.

In [6]:
# Google drive에서 받기 위해 gdown을 설치합니다
!pip install gdown
# maestro-npy.zip 파일을 다운로드 합니다.
!gdown https://drive.google.com/uc?id=1PCsy4C0XdO2hWedw7w6FZ39neCgAdNAi
# maestro-npy.zip 압축파일을 풉니다.
!unzip -o -qq maestro-npy.zip

Downloading...
From: https://drive.google.com/uc?id=1PCsy4C0XdO2hWedw7w6FZ39neCgAdNAi
To: /home/gaudio/ste/projects/lectures/AI-for-Music/source/maestro-npy.zip
100%|████████████████████████████████████████| 124M/124M [00:32<00:00, 3.81MB/s]


In [7]:
import os
import numpy as np
from torch.utils.data import Dataset, DataLoader

### Dataset에 관련된 hyper-parameters를 정의합니다.

In [8]:
from easydict import EasyDict
# 각 이벤트 종류 (interval, velocity 등)에 대해 사용할 token의 갯수를 지정합니다.
dims = EasyDict(interval = 100,
                velocity = 32,
                note_on = 128,
                note_off = 128,
                pedal_on = 1,
                pedal_off = 1)

# 각 이벤트 종류 (interval, velocity 등)에 대해 token의 offset을 지정합니다.
offsets = EasyDict(interval = 100,
                   velocity = dims.interval,
                   note_on = dims.interval + dims.velocity,
                   note_off = dims.interval + dims.velocity + dims.note_on,
                   pedal_on = dims.interval + dims.velocity + dims.note_on + dims.note_off,
                   pedal_off = dims.interval + dims.velocity + dims.note_on + dims.note_off + dims.pedal_on)

# Dataset에 사용될 hyper-parameters를 지정합니다.
dataset_hparams = EasyDict(root_dir = 'dataset/',
                           max_note_duration = 2, # seconds
                           token_length = 1024,
                           dims = dims,
                           offsets = offsets
                          )
for hp in dataset_hparams:
    print(hp, dataset_hparams[hp])

root_dir dataset/
max_note_duration 2
token_length 1024
dims {'interval': 100, 'velocity': 32, 'note_on': 128, 'note_off': 128, 'pedal_on': 1, 'pedal_off': 1}
offsets {'interval': 100, 'velocity': 100, 'note_on': 132, 'note_off': 260, 'pedal_on': 388, 'pedal_off': 389}


### Pytorch framework에서 사용하는 규격대로 Dataset class를 정의합니다.

In [10]:
from midi_utils import event_list_to_tokens

class MaestroDataset(Dataset):
    def __init__(self, dataset_hparams):
        super().__init__()
        self.hp = dataset_hparams
        # .npy 파일의 목록을 저장합니다.
        self.files = [os.path.join(self.hp.root_dir, file) for file in os.listdir(self.hp.root_dir) if 'npy' in file]
    
    # 전체 data 갯수를 반환합니다.
    def __len__(self):
        return len(self.files)
    
    # 하나의 data index를 받아 token sequence를 반환합니다.
    def __getitem__(self, index):
        # file 경로를 구합니다.
        file = self.files[index]
        
        # .npy 파일을 읽어 event_list를 만듭니다.
        event_list = np.load(file, allow_pickle=True)
        
        # event_list를 token sequence로 변환합니다.
        tokens = event_list_to_tokens(event_list, self.hp)
        
        # dataset에 사용될 만큼 token sequence를 자릅니다.
        if len(tokens) < self.hp.token_length:
            start_index = 0
        else:
            start_index = np.random.randint(0, len(tokens)-self.hp.token_length)
        tokens = np.array(tokens[start_index:start_index+self.hp.token_length])
        
        # token sequence가 dataset에 사용될 길이보다 짧은 경우 padding을 합니다.
        tokens_padded = np.zeros(self.hp.token_length, dtype=np.int)
        tokens_padded[:len(tokens)] = tokens
        
        # padding된 token sequence를 반환합니다.
        return np.array(tokens_padded)

### Pytorch에서 사용할 수 있도록 dataset과 train_loader를 만듭니다.

In [11]:
dataset = MaestroDataset(dataset_hparams)
print(dataset)

train_loader = DataLoader(dataset, batch_size=16)
print(train_loader)

In [12]:
for batch in train_loader:
    print(batch.shape)
    for i, data in enumerate(batch):
        print(str(i) + '번 token sequence :', data.numpy())
    break

torch.Size([16, 1024])
0번 token sequence : [205   0 114 ...   1 333   0]
1번 token sequence : [  1 327   0 ...   1 388   0]
2번 token sequence : [ 22 342   0 ... 109 188   5]
3번 token sequence : [325   1 121 ...   4 344   0]
4번 token sequence : [  0 297   1 ... 311   1 323]
5번 token sequence : [201   2 329 ... 196   0 389]
6번 token sequence : [115 218   1 ... 117 221   0]
7번 token sequence : [196   1 321 ... 111 217  16]
8번 token sequence : [189   1 329 ...   0 109 184]
9번 token sequence : [  0 388   1 ... 189   0 320]
10번 token sequence : [117 204   1 ... 122 182   0]
11번 token sequence : [192   2 320 ...   0 337  13]
12번 token sequence : [233   1 360 ...   4 110 201]
13번 token sequence : [  0 117 207 ...   0 114 179]
14번 token sequence : [  0 324   0 ... 184   0 327]
15번 token sequence : [172   0 115 ...   0 329   9]


In [23]:
print(*data.data.cpu().numpy()[:20], '...', sep=', ')

172, 0, 115, 184, 1, 111, 194, 0, 312, 0, 328, 0, 300, 1, 322, 1, 111, 192, 1, 320, ...
